In [1]:
#Modules used
import requests
import pandas as pd
import time as t
import urllib.request
import os.path
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

#Read in the cphea chemical list as a dataframe
file = r"C:\Users\mmille16\OneDrive - Environmental Protection Agency (EPA)\Profile\Downloads\pprtv_cphea_chemicals_20230123.xlsx"
cphea = pd.read_excel(file)

In [4]:
#Holds info about problematic 
retry = {}

#Save location of origin pdfs
save_path = r"L:\PRIV\ToxValDB\Document Webscrape\pprtv_cphea\PPRTV CPHEA\Origin Documents"

for i in range(len(cphea)):
#Necessary names for each chemical
    chem_name = cphea.loc[i,'Chemical']
    file_name = f'{save_path}/{chem_name}_pprtv_cphea_origin.pdf' #Save name of support pdf
    if os.path.isfile(file_name): #In case this needs to be run again, make sure I don't double download a file
        continue
    #Extract the pdf url from the excel file
    url = cphea.loc[i,'PPRTV Assessment']
    try:
        t.sleep(1)
    #Access the pdf url, open a file in the pprtv cphea pdf folder, write, and save the file from the pdf url
        response = urllib.request.urlopen(url)
    except Exception as e: #In case the pdf url is invalid, save some info about it please and thank you
        retry[cphea.loc[i,'PPRTV Substance ID']] = chem_name
        continue
#Open and save the pdf in the save path
    file = open(file_name,'wb')
    file.write(response.read())
    file.close()

#Might be able to find more valid pprtv pdf links on the pprtv_chpea webpage, but lets only do that if necessary
print('retrying origin documents not found before')
for sub_id in retry:
    file_name = f'{save_path}/{retry[sub_id]}_pprtv_cphea_origin.pdf'
    if os.path.isfile(file_name): #In case this needs to be run again, make sure I don't double download a file
        continue
#Slenium browser settings
    browser = f'https://cfpub.epa.gov/ncea/pprtv/chemicalLanding.cfm?pprtv_sub_id={sub_id}'
    Options = webdriver.ChromeOptions()
    Options.headless = True
    s=Service(r'C:\Users\mmille16\Documents\chromedriver.exe')
    driver = webdriver.Chrome(service=s, options = Options)
    driver.get(browser)
    
#Look for the webpage element with the pprtv link
    try:
        item = driver.find_element(By.PARTIAL_LINK_TEXT,'Toxicity')
    except:
#Some may not have one. Tell me just in case
        print(f'{retry[sub_id]} webpage does not have a link to the pprtv pdf')
        continue
#Travel through the webpage until we arrive at the downloadable pdf link
    item.click()
    t.sleep(0.5)
    tab = driver.find_element(By.PARTIAL_LINK_TEXT,'Downloads')
    tab.click()
    t.sleep(0.5)
#Get the pdf url and save as a pdf in the directory
    doc = driver.find_element(By.PARTIAL_LINK_TEXT,'Toxicity')
    response = urllib.request.urlopen(doc.get_attribute('href'))
#Open and save the pdf file in the folder directory
    file = open(file_name, 'wb')
    file.write(response.read())
    file.close()
    driver.close()
print('done')

retrying origin documents not found before
p-Chloro-m-cresol webpage does not have a link to the pprtv pdf
2-Naphthylamine webpage does not have a link to the pprtv pdf
Sodium acid pyrophosphate webpage does not have a link to the pprtv pdf
done
